# Example use of the Sonic module Nodosus.jl

In [ ]:
import Pkg                      # package manager
Pkg.activate("..")              # Project.toml
Pkg.instantiate()               # https://julialang.github.io/Pkg.jl/v1.0/index.html

In [ ]:
import Dates, JuliaDB, Plots, Random

In [ ]:
include("../src/Linckii.jl")
include("../src/LinckiiSecrets.jl")
include("../src/Nodosus.jl")

Get access token,

In [ ]:
@time access = Linckii.get_access(LinckiiSecrets.keab)

Load site,

In [ ]:
@time nodes, devices, sensors = Linckii.loadsite(access)

Get side 1 substation series,

In [ ]:
sensor_names = (
    :meter_effect,
    :meter_primreturntemp,
    :meter_primsupplytemp,
    :meter_volumeflow,
)

In [ ]:
signals = JuliaDB.reindex(
    JuliaDB.join(
        Linckii.flatten_nodes(JuliaDB.filter(r -> r.device_id == 4, nodes)),
        JuliaDB.filter(r -> r.sensor_name in sensor_names, sensors);
        lkey = :sensor_id,
        rkey = :sensor_id,
    ),
    (:node_id, :sensor_id),
)

In [ ]:
signal_rows = JuliaDB.rows(signals)
length(signal_rows)

Load data,

In [ ]:
p, q = Dates.Minutes(10), Dates.Minutes(120)

In [ ]:
pkey = (:node_id, :datetime)

In [ ]:
tables = ()
@time for r in signal_rows
    filepath = "$(Linckii.datapath(access, r.node_id, r.sensor_name)).db"
    try
        data = Linckii.loaddata(access, r.node_id, r.sensor_name)
        pkeys = JuliaDB.pkeys(data)
        push!(
            JuliaDB.transform(
                JuliaDB.table(Nodosus.pp1(JuliaDB.rows(data), p, q), pkey = (:node_id, pkeys...)),
                :node_id => r.node_id,
                :value => r.sensor_name,
            ),
            tables,
        )
        println("$(filepath): Done")
    catch e
        println("$(filepath): $(e)")
    end
end
data = reduce((l, r) -> JuliaDB.join(l, r; lkey = pkey, rkey = pkey, how = :outer), tables)

Save data,

In [ ]:
@time JuliaDB.save(
    data,
    "$(Linckii.datapath(access)).db",
)

Load data,

In [ ]:
@time data = JuliaDB.load(
    "$(Linckii.datapath(access)).db",
)

In [ ]:
# two plots, one for a random node but both supply and return temp, and one for all nodes but only their difference

Complex plot,

In [ ]:
t = JuliaDB.filter(
    r -> r.datetime >= Dates.DateTime(2018) && r.datetime <= Dates.DateTime(2019),
    data,
)
x = JuliaDB.select(t, :datetime)
y = JuliaDB.select(t, :meter_primreturntemp)
Plots.plot(x, y, xlabel = :datetime, label = Linckii.datapath(access, r.node_id, r.sensor_name))